In [1]:
import os
import glob
import shutil

from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [2]:
try:
    with open('../gem_apikey.txt') as f:
        api_key = f.read().strip()
        print("Info: Archivo '../gem_apikey.txt'. OK.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'. Asegúrate de que la ruta es correcta.")
    api_key = None

Info: Archivo '../gem_apikey.txt'. OK.


In [ ]:
def process_and_store_pdf_in_vectordb(pdf_path, FOLDER_INDEX_base):
    """
    Lee un único documento PDF, crea su base de datos vectorial (FAISS) 
    y la guarda en disco.
    
    Returns:
        bool: True si el proceso fue exitoso, False si hubo un error.
    """
    if not api_key:
        print(f"Proceso detenido para '{pdf_path}'. No se pudo cargar la API key de Google.")
        return False    

    file_name = os.path.basename(pdf_path)
    base_name, _ = os.path.splitext(file_name)
    output_path = os.path.join(FOLDER_INDEX_base, f"faiss_index_{base_name}")
    
    print("-" * 50)
    print(f"Iniciando procesamiento para: '{file_name}'")

    try:
        # Aquí va tu lógica real de LangChain, PyPDFLoader, FAISS, etc.
        
        # 1. Carga del documento PDF
        print(f"Cargando desde: {pdf_path}")
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        if not documents:
            raise ValueError("El PDF está vacío o no se pudo leer.")
        
        # 2. División del texto en fragmentos
        print("Dividiendo en fragmentos...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = text_splitter.split_documents(documents)

        # 3. Creación de Embeddings y la base de datos vectorial (en memoria)
        print("Creando embeddings con Gemini y guardando en FAISS...")
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
        vectorstore = FAISS.from_documents(texts, embeddings)
        print("Base de datos vectorial creada en memoria.")


        # 4. GUARDADO EN DISCO de la base de datos vectorial
        vectorstore.save_local(output_path)
        print(f"¡Éxito! Índice guardado en: '{output_path}'")
        
        return True

    except Exception as e:
        print(f"ERROR al procesar el archivo {os.path.basename(pdf_path)}: {e}")
        return False

In [ ]:
def main():
    """
    Función principal que orquesta el pipeline de datos.
    Busca PDFs en 'raw', los procesa y los mueve a 'processed' (si éxito)
    o a 'error' (si falla).
    """
    # --- CONFIGURACIÓN DE LA ESTRUCTURA DE CARPETAS FINAL ---
    FOLDER_BASE = "../database"
    FOLDER_RAW = os.path.join(FOLDER_BASE, "raw")
    FOLDER_PROCESSED = os.path.join(FOLDER_BASE, "processed")
    FOLDER_ERROR = os.path.join(FOLDER_BASE, "error") # <-- Nueva carpeta
    FOLDER_INDEX = os.path.join(FOLDER_BASE, "indexed", "faiss")

    # Crear toda la estructura de carpetas si no existe
    for path in [FOLDER_RAW, FOLDER_PROCESSED, FOLDER_ERROR, FOLDER_INDEX]:
        os.makedirs(path, exist_ok=True)

    # 1. Buscar nuevos documentos en la carpeta RAW
    documents_to_process = glob.glob(os.path.join(FOLDER_RAW, "*.pdf"))

    if not documents_to_process:
        print("No hay nuevos documentos en 'Database/raw' para procesar.")
        return

    print(f"Se encontraron {len(documents_to_process)} documentos para procesar.")
    
    # 2. Procesar cada documento encontrado
    for source_path in documents_to_process:
        file_name = os.path.basename(source_path)
        
        # Llama a la función de procesamiento
        is_successful = process_and_store_pdf_in_vectordb(source_path, FOLDER_INDEX)

        # 3. Mover el archivo a la carpeta correspondiente ('processed' o 'error')
        if is_successful:
            print(f"Moviendo '{file_name}' a la carpeta de procesados...")
            destination_path = os.path.join(FOLDER_PROCESSED, file_name)
        else:
            print(f"Moviendo '{file_name}' a la carpeta de errores...")
            destination_path = os.path.join(FOLDER_ERROR, file_name)
        
        try:
            shutil.move(source_path, destination_path)
            print(f"-> Movido exitosamente a '{destination_path}'")
        except Exception as e:
            print(f"ERROR CRÍTICO: No se pudo mover el archivo '{file_name}'. Error: {e}")

    print("-" * 50)
    print("\nProceso de ingesta de documentos completado.")


if __name__ == "__main__":
    main()

Se encontraron 3 documentos para procesar.
--------------------------------------------------
Iniciando procesamiento para: 'sistema-cardiovascular.pdf'
Cargando desde: ../database\raw\sistema-cardiovascular.pdf
Dividiendo en fragmentos...
Creando embeddings con Gemini y guardando en FAISS...
Base de datos vectorial creada en memoria.
¡Éxito! Índice guardado en: '../database\indexed\faiss\faiss_index_sistema-cardiovascular'
Moviendo 'sistema-cardiovascular.pdf' a la carpeta de procesados...
-> Movido exitosamente a '../database\processed\sistema-cardiovascular.pdf'
--------------------------------------------------
Iniciando procesamiento para: 'sistema-nervioso-central.pdf'
Cargando desde: ../database\raw\sistema-nervioso-central.pdf
Dividiendo en fragmentos...
Creando embeddings con Gemini y guardando en FAISS...
Base de datos vectorial creada en memoria.
¡Éxito! Índice guardado en: '../database\indexed\faiss\faiss_index_sistema-nervioso-central'
Moviendo 'sistema-nervioso-central.pd